# Newspaper metadata: exploration & sampling

This notebook illustrates ways to access, filter, join and sample from a set of three CSV tables containing historical newspaper metadata released by the Living with Machines (LwM) project.

Newspaper metadata refers to information *about* the titles, issues and articles that were published, such as their place and date of publication (as opposed to the newspaper content itself). By exploring this metadata, we can identify subsets of the newspaper collections of particular interest for a given research question. Given the large volume of textual data in the newspaper collections themselves, this process of sampling is an important step to focus subsequent analysis.

In the final section of the notebook, a sample of metadata will be used to download the corresponding article text.

**NOTE:** This is an interactive notebook. To work properly, the cells should be executed in order from top to bottom. Most of the cells will work automatically, but sometimes some manual intervention is needed (e.g. an exercise). These are highlighted with the the words "**YOUR TURN**".

#### Contents:

1. Newspaper metadata
1. Data download
1. Read CSV files
1. Summary statistics
1. Construct samples
   - Filtered samples
   - Impartial samples
   - Prescriptive samples
1. Full text access


In [ ]:
import os
import re
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', 4)
%load_ext dotenv
%dotenv

## 1. Newspaper metadata

This notebook focusses primarily on data extracted from the LwM relational database of historical newspaper metadata ([lwmdb](https://github.com/Living-with-machines/lwmdb)).

These extracts consist of three tables, containing one row per newspaper title, issue and item (respectively):

| Table  | Rows        | Columns | Size   |
|--------|-------------|---------|--------|
| TITLES | 1,504       | 35      | 430 KB |
| ISSUES | 2,425,752   | 6       | 147 MB |
| ITEMS  | 223,012,941 | 7       | 15 GB |

The term "item" refers to a semantic unit within a newspaper page (identified during the digitisation process). Typically these are newspaper articles but they may instead be adverts or illustrations. 

Due to their large size, the number of columns in the ISSUES and ITEMS tables is minimised. Additional metadata columns can be added by joining with the TITLES table (as demonstrated below).

At 6.7 GB, the ITEMS table remains too large to work with conveniently on most personal computers. For this reason, we shall restrict our attention to a subset of newspaper items, which we refer to as "open access" because they are not subject to licensing rights which limit access to the newspaper content.

| Table              | Rows      | Columns | Size   |
|--------------------|-----------|---------|--------|
| OPEN ACCESS ITEMS  | 9,326,170 | 7       | 616 MB |


## 2. Data download

This step only needs to be performed once, after which the downloaded metadata will be available in the `./data` subdirectory.

The CSV files will be downloaded and stored locally in the following locations:

In [ ]:
titles_csv = "./data/title_query_sorted.csv"
issues_csv = "./data/issue_query_sorted.csv"
items_csv = "./data/item_query_oa_sorted.csv"

The largest CSV files are zipped in the remote storage account and will be stored as:

In [ ]:
issues_zip = "./data/issue_query_sorted.zip"
items_zip = "./data/item_query_oa_sorted.zip"

**NOTE:** Due to the large size of the ITEMS table we will work with a subset of the data, including only the "open access" material. 

<details>
  <summary>Click here for instructions on how to obtain the complete ITEMS table.</summary>
  
To work with the complete ITEMS table, replace the file paths in the preceding two cells with the following values:
```
items_csv = "./data/item_query_sorted.csv"
items_zip = "./data/item_query_sorted.zip"
```
Note, however, that the complete ITEMS CSV file is 15 GB. This is too large to fit into the working memory of most personal computers. Instead it can be read and processed in chunks using the `chunksize` parameter in [`pandas.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html).
</details>

#### Check for locally-stored data

If all of the data files are already available locally, you can skip forward to [Section 3](#readcsvfiles) to read the CSV files.

Otherwise, execute the cells below to download the data.

In [ ]:
all_files = [titles_csv, issues_csv, items_csv, issues_zip, items_zip]
if all(map(os.path.exists, all_files)):
    print("All data files found. Skip forward to Section 3.")
else:
    print("One or more data files were not found. Run the following cells to download data.")

#### Install AzCopy

To download files from Azure cloud storage we will need the AzCopy command line tool.

The following cell checks whether AzCopy is already installed. If it is not, a suitable version can be found [here](https://learn.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10#download-azcopy).

In [ ]:
if not all(map(os.path.exists, all_files)):
    azcopy_version = !azcopy --version
    if "command not found" in azcopy_version[0]:
        print("Please install AzCopy using the link above.")
    else:
        print("AzCopy is already installed.")

#### Establish access to Azure blob storage

To download newspaper metadata files you will need a "SAS token" (shared access signature) granting read-only access to the project's Azure cloud storage.

The SAS token is a string of characters (similar to a password) and will be provided by the workshop organisers. 

**YOUR TURN:** To make the SAS token accessible, create a new file named `.env` in the same folder as this notebook. This file must contain the following line (with the provided SAS token between the quotes):
 ```
SAS_TOKEN="COPY_SAS_TOKEN_HERE"
 ```

**NOTE:** After editing the `.env` file you will need to restart the Jupyter kernel and re-execute all of the code cells up to this point.

The next cell checks that a valid SAS token has been found. 

In [ ]:
sas_token = os.environ.get("SAS_TOKEN")
if not sas_token:
    print("No SAS token found. Please follow the instructions above this cell.")
else:
    matched = re.match(r"^sv=.*se=(?P<expiry>20[0-9]{2}-[0-1][0-9]-[0-9]{2}).*sig=", sas_token) 
    if not matched or not "expiry" in matched.groupdict():
        print("Invalid SAS token.")
    else:
        print(f"SAS token found. Expiry date: {matched["expiry"]}")

#### Copy the data from Azure blob storage

Download the TITLES table:

In [ ]:
if not os.path.exists(titles_csv):
    path = f"'https://opennewspapers.blob.core.windows.net/lwmdb/title_query_sorted.csv?{sas_token}'"
    !azcopy copy {path} {titles_csv}

Download and unzip the ISSUES table:

In [ ]:
if not os.path.exists(issues_zip):
    path = f"'https://opennewspapers.blob.core.windows.net/lwmdb/issue_query_sorted.zip?{sas_token}'"
    !azcopy copy {path} {issues_zip}
if not os.path.exists (issues_csv):
    !unzip -j {issues_zip} -d "./data"

Download and unzip the ITEMS table:

In [ ]:
if not os.path.exists(items_zip):
    path = f"'https://opennewspapers.blob.core.windows.net/lwmdb/{items_zip.split('/')[-1]}?{sas_token}'"
    !azcopy copy {path} {items_zip}
if not os.path.exists (items_csv):
    !unzip -j {items_zip} -d "./data"

#### Check the downloaded data

If the downloads were successful, the CSV files will now be available in the `data/` subdirectory.

In [ ]:
!ls -lh ./data/*.csv
if all(map(os.path.exists, (titles_csv, issues_csv, items_csv))):
    print("All CSV files found!")

**YOUR TURN: (Optional)**

The `data/` subdirectory should now contain three CSV files: `title_query_sorted.csv`, `issue_query_sorted.csv` and `item_query_sorted.csv`

In the next section we will read these tables as a [`pandas`](https://pandas.pydata.org/) data frames. 

But you might also want to view them in a spreadsheet application, such as Microsoft Excel or OpenOffice Calc. In that case, you will need to use the "Import" function, to make sure the column separators are interpreted correctly.

To view the table in Excel, go to the `File` menu and choose `Import`. Select the type `CSV file`, then navigate to the `data/` subdirectory and select the file named `title_query_sorted.csv`. 

On the next step, choose `Delimited` and on the following step select `Other` and enter the pipe character '|' in the box. Then click `Finish`.


To view it in OpenOffice... TODO.

## 3. Read CSV files
<a id='readcsvfiles'></a>

Now that we have a local copy of the CSV files, we are ready to read in the data.

We begin by defining a function to read each of the CSV files with the correct column types.

In [ ]:
def read_csv(file, int_cols=list(), float_cols=list(), date_cols=list(), **kwargs):
    df = pd.read_csv(file, dtype=str, **kwargs)
    # Remove whitespace from column names:
    df.columns = df.columns.str.replace(' ', '')
    for arg in int_cols:
        # Replace whitespace strings with NaN and convert to Int64:
        df[arg] = df[arg].str.replace(' ', '').replace('', np.nan).astype('Int64')
    for arg in float_cols:
        df[arg] = pd.to_numeric(df[arg], errors='coerce')
    for arg in date_cols:
        df[arg] = pd.to_datetime(df[arg])
    return df 

#### Read the TITLES table:

In [ ]:
titles = read_csv(titles_csv, float_cols=['latitude', 'longitude'], delimiter='|')
display(titles)

#### Read the ISSUES table:

In [ ]:
issues = read_csv(issues_csv, date_cols=["date"], delimiter='|')
display(issues)

#### Read the ITEMS table:

**Note:** the complete ITEMS table contains 223,012,941 rows and is 15 GB in size, too large for most personal computers to store in memory. For this reason, we restrict our attention to the "open access" material. This subset of the ITEMS table contains 9,326,170 rows.

In [ ]:
items = read_csv(items_csv, int_cols=['word_count'], float_cols=['ocr_quality_mean'], date_cols=["date"], delimiter='|')
display(items)

## 4. Summary statistics

To better understand the data, we'll calculate some summary statistics.

#### Data Providers

In the TITLES and ISSUES tables, the `data_provider` refers to the organisation that provided access to the digitised newspapers collections.

There are four data providers:
 - **FindMyPast**: a private company which owns the rights to the [British Newspaper Archive](https://www.britishnewspaperarchive.co.uk/).
 - **JISC**: a not-for-profit organisation that funded two phases of newspaper digitisation between 2004 and 2009.
 - **Living with Machines**: a collaborative research project involving the British Library which ran from 2018 to 2023.
 - **Heritage Made Digital**: a British Library digitisation programme.

This is an important variable in our data, because it determines the level of accessibility of the newspaper content. This content may include page images and/or text obtained from the images via Optical Character Recognition (OCR) software.

Only material from the last two data providers, Living with Machines and Heritage Made Digital, is openly accessible without subscription or special agreement.

So let's begin by counting the number of newspaper titles and issues, separately for each of the data providers:

In [ ]:
title_count = titles['data_provider'].value_counts().to_frame(name = "Number of Titles")
issue_count = issues['data_provider'].value_counts().to_frame(name = "Number of Issues")
data_providers = pd.concat([title_count, issue_count], axis = 1)
data_providers.reset_index(inplace=True, names = "Data Provider")
data_providers

We see that the open access material includes 107,712 issues from 121 titles (although it is only ~5% of the whole).

#### Temporal distribution

The ISSUES table includes the date of publication, ranging from 1720-05-02 to 1957-12-31.

Here we plot a histogram showing the number of issues published per decade, from 1750 to 1950.

In [ ]:
bins = list(range(1750, 1960, 10))
plt.hist([date.year for date in issues['date']], bins=bins, rwidth=0.88, edgecolor='black')
plt.title('Number of newspaper issues, by decade');

**YOUR TURN: (Optional)**

Plot a histogram, similar to the one above, of the temporal distribution of (open access) newspaper **items**.

<details>
  <summary>Click here for a hint.</summary>
  
**Hint:**

Copy the code in the preceding cell and paste it into a new cell below this one. 

Edit the code so that it refers to the table named `items`, instead of `issues`.

Then change the histogram title inside the `plt.title` command.
</details>

In [ ]:
bins = list(range(1750, 1960, 10))
plt.hist([date.year for date in items['date']], bins=bins, rwidth=0.88, edgecolor='black')
plt.title('Number of OA newspaper items, by decade');

## 5. Construct samples

This section contains some examples of how to create subsamples of the newspaper metadata. Three types of sample are considered:
1. **Filtered samples**, obtained via simple filtering with respect to the values of one or more variables.
2. **Impartial samples**, obtained by random selection which preserves multivariate distributions between variables.
3. **Prescriptive samples**, obtained by random selection while prescribing a desired distribution with respect to particular variables.

### Filtered samples

We begin with some simple examples of filtering the table of newspaper issues by time period and/or place of publication.

First we filter on the `date` column to obtain all of the newspaper issues from the 1850s:

In [ ]:
issues_1850s = issues.query('18500101 <= date < 18591231').sort_values('date')
display(issues_1850s)

The filtered tables take up much less space in memory, so we can join it to the TITLES table to add additional columns of metadata.

To do this, we use the [`pd.merge`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) function, and join on the three columns that are common to both tables: the `publication_code`, `newspaper_id` and `data_provider`.

In [ ]:
issues_1850s = pd.merge(issues_1850s, titles, how='left', on=['publication_code', 'newspaper_id', 'data_provider'])
display(issues_1850s)

In the next example, we filter on the `historic_county_label` column to obtain all of the issues from the historic county of Dorset:

In [ ]:
titles_dorset = titles[titles['historic_county_label'] == 'Dorset']
issues_dorset = issues[issues['publication_code'].isin(titles_dorset['publication_code'])]
issues_dorset = pd.merge(issues_dorset, titles, how='left', on=['publication_code', 'newspaper_id', 'data_provider'])
display(issues_dorset)


Combining the last two examples, we obtain all of the newspaper issues from the historic county of Dorset published during the 1850s:

In [ ]:
issues_dorset_1850s = issues_dorset.query('18500101 <= date < 18591231').sort_values('date')
display(issues_dorset_1850s)

The next example filters on the `data_provider` column to produce a table of open access newspaper issues. As explained above, these are the newspapers for which the data provider is either `Living with Machines` or `Heritage Made Digital`.

In [ ]:
# All open access newspaper issues:
oa_data_providers = ('Living with Machines', 'Heritage Made Digital')
issues_oa = issues[issues['data_provider'].isin(oa_data_providers)]
issues_oa = pd.merge(issues_oa, titles, how='left', on=['publication_code', 'newspaper_id', 'data_provider'])

# Insert a new column for the year of publication:
issues_oa.insert(3, "year", [d.year for d in issues_oa['date']])
display(issues_oa)

### Random samples

Next we'll create some random samples using the [`subsamplr`](https://github.com/Living-with-machines/subsamplr) tool, which makes it easy to construct samples that either reflect the underlying data, or prescribe the joint distributions of particular variables of interest.

First we install `subsamplr` from GitHub:

In [ ]:
%pip install --upgrade -q git+https://github.com/Living-with-machines/subsamplr@main

Next we specify a set of variables to be used for sampling (referred to as "dimensions").

As an example, we shall construct subsamples from the table of open access newspaper issues, taking into account the `year` and `political_leaning_label` variables.

The `year` is a discrete variable. We must specify its type (integer), the discretisation size (one in this case), the variable range of interest (i.e. min & max) and the "bin size", which refers to the size of the bins into which all of the table rows will be assigned. The sampling procedure involves the random selection of a bin followed by the random selection of a row inside the bin. Here we specify a bin size of 10, meaning that each bin corresponds to a decade during the `year` range.

The `political_leaning_label` is a categorical variable. We must specify its type (string) and the list of categories of interest.

Any table rows whose values fall outside of the specified range (for a discrete variable) or list of categories (for a categorical variable) will be ignored when drawing the sample. For instance, here we have omitted the `independent` political leaning from the list of categories, so no articles from independent-leaning newspapers will be selected.

**Note:** In addition to discrete and categorical variables, `subsamplr` also supports continuous numerical variables (for example, see the `ocr_quality_mean` variable defined [here](https://github.com/Living-with-machines/subsamplr/blob/main/examples/example_database_subsampling.ipynb)).

In [ ]:
import yaml
from io import StringIO
config_str = """
variables:
    - {name: 'year', class: 'discrete', type: 'int', min: 1850,
        max: 1899, discretisation: 1, bin_size: 10}
    - {name: 'political_leaning_label', class: 'categorical', type: 'str',
        categories: ['liberal', 'neutral', 'conservative']}
"""
config = yaml.safe_load(StringIO(config_str))

Next we construct the collection of bins, given the above configuration, and assign the table rows (or sampling "units") to those bins.

We identify each row (or "unit") by the `issue_id` variable, which is a unique identifier for the rows in the `issues_oa` table.

In [ ]:
from subsamplr import BinCollection, UnitGenerator
bins = BinCollection.construct(config, track_exclusions=True)
generator = UnitGenerator.generate_units(
        issues_oa, unit_id="issue_id", variables=bins.dimensions)
units = list(generator)
for unit, values in units:
    bins.assign_to_bin(unit, values)

To summarise the binning process, we had 107,712 rows in our original table of open access newspaper issues. From those rows 47,355 sampling units were constructed. The other 60,357 had missing values in the `political_leaning_label` column.

Based on the configuration above, 28,708 have been assigned to bins for sampling, and the other 18,647 were excluded because their values fall outside of the specified variable ranges.

In [ ]:
print(f"Open access issues: \t{len(issues_oa)}")
print(f"Sampling units: \t{len(units)}")
print(f"Binned: \t\t{bins.count_units()}")
print(f"Excluded: \t\t{bins.count_exclusions()}")

#### Impartial samples

Before drawing a random sample, it is advisable to set a random seed. The seed is an arbitrary number used to initialise the random number generator in [`numpy`](https://numpy.org/), so that an identical sample can be generated in future by specifying the same seed.

In [ ]:
seed = 654321
from numpy.random import seed as npseed
npseed(seed)

Drawing samples from our populated collection of bins is straightforward. Here we extract a sample of 2000 open access newpspaper issues:

In [ ]:
k = 2000
sample_units = bins.select_units(k)
issues_sample = issues_oa[issues_oa['issue_id'].isin(sample_units)]
display(issues_sample)

The political leanings in the sample are similar to those in the full table of open access issues:

In [ ]:
print(issues_sample['political_leaning_label'].value_counts().reindex().to_string())

In [ ]:
print(issues_oa['political_leaning_label'].value_counts(dropna=False).reindex().to_string())

#### Prescriptive samples

 Now we construct a sample in which the distributions of the configured variables are deliberately controlled.

The specified range of values for the `year` variable (1850-1859) is split across five bins, one for each decade.

The specified range of values for the `political_leaning_label` variable is split across three categories: 'liberal', 'neutral' and 'conservative'.

We must prescribe (relative) weights for each variable and for each bin. Therefore we define our `weights` as a tuple of the same length as `bins.dimensions`, each element being itself a list of weights, one for each bin.

Here we weight the last two decades twice as heavily as the first three decades, and we assign equal weight to all three political leaning categories.

In [ ]:
year_weights = [1, 1, 1, 2, 2]
political_leaning_label_weights = [1, 1, 1]
weights = (year_weights, political_leaning_label_weights)

Once again we set the random seed (for reproducibility), and then call `select_units()` to generate the sample, this time passing in the weights.

In [ ]:
npseed(seed)
sample_units = bins.select_units(k, weights=weights)
issues_sample = issues_oa[issues_oa['issue_id'].isin(sample_units)]
display(issues_sample)

This time all three political leanings are represented approximately equally, because the sample was selected according to the prescribed uniform distribution on that dimension.

In [ ]:
print(issues_sample['political_leaning_label'].value_counts().reindex().to_string())

On the time axis, the weights were skewed towards to last two decades of the century.

In [ ]:
plt.hist(issues_sample['year'], bins=[1850, 1860, 1870, 1880, 1890, 1900], rwidth=0.88, edgecolor='black')
plt.title('Issue counts in a prescriptive sample, by decade');

## 6. Full text access

### Download full text

The following helper functions will be used to download zip archives containing the full text of all articles for a given newspaper title (identified by its `publication_code`).

In [ ]:
# Gets a short acronym for the data provider for a given publication.
def provider(publication_code):
    data_provider = titles[titles.publication_code == publication_code].data_provider.item()
    return ''.join([w[0] for w in data_provider.split()]).lower()

# Gets the relative path to the article full text (zip file) for a given publication.
def fulltext_zip(publication_code):
    return f"./data/{provider(publication_code)}-alto2txt/{publication_code}_plaintext.zip"

# Returns the could storage location of the article full text (zip file) for a given publication.
def fulltext_blob(publication_code):
    stub = 'https://opennewspapers.blob.core.windows.net/alto2txt/'
    return f"'{stub}{provider(publication_code)}-alto2txt/plaintext/{publication_code}_plaintext.zip?{sas_token}'"

# Downloads a zip file containing article full text for a given collection of publications.
def download_fulltext(publication_codes):
    for pub in publication_codes:
        if os.path.exists(fulltext_zip(pub)):
            continue
        !azcopy copy {fulltext_blob(pub)} {fulltext_zip(pub)}

download_fulltext(['0002083'])

### Extract article text

TODO.

In [ ]:

def article_txt()

def extract_fulltext(issues):
    for issue in issues:
        if os.path.exists()
    